In [ ]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [ ]:
os.environ["GROQ_API_KEY"] = ""

In [ ]:
loader = DirectoryLoader('/content/drive/MyDrive/Mayo_LT', glob="**/*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
type(texts)

list

In [ ]:
len(texts)

335

In [ ]:
#texts[:3]

In [ ]:
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
persist_directory = "liver_transplant_db"

In [ ]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
# retriever
retriever = vectordb.as_retriever()

In [ ]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [ ]:
# Define the custom prompt template
custom_prompt_template = """
You are an expert liver transplant coordinator with access to Mayo Clinic's guidelines and policies on liver transplantation. Answer the following question comprehensively and with precision based on the provided context. Provide detailed explanations and, when applicable, include numerical criteria and eligibility thresholds. Always cite the specific guideline or policy you're referencing.

Query: {question}

Context: {context}

Please structure your response as follows:
1. Comprehensive answer to the query (provide a detailed explanation, including background information if relevant)
2. Key points or criteria (always include numerical data if applicable, but don't force it if not relevant)
3. Relevant guidelines or policies (mention specific documents and provide brief summaries of their content)
4. Additional considerations or implications

Ensure your response is tailored to the specific query. Not all sections may be necessary for every question - use your judgment to provide the most relevant and helpful information.

After formulating your response, please review it for accuracy, especially regarding any numerical data or specific guidelines mentioned.
"""

# Create a PromptTemplate
PROMPT = PromptTemplate(
    template=custom_prompt_template, input_variables=["context", "question"]
)


In [ ]:
# Create a QA chain with the custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)



In [ ]:
# invoke the qa chain and get a response for user query
query = "What is the liver transplant eligibility criteria for intrahepatic cholangiocarcinoma at Mayo Clinic?"
response = qa_chain.invoke({"query": query})
print(response["result"])

**Comprehensive Answer**

The liver transplant eligibility criteria for intrahepatic cholangiocarcinoma (iCCA) at Mayo Clinic are outlined in the Liver Transplant for Patients with Intrahepatic Cholangiocarcinoma (iCCA) Guideline. However, it is essential to note that patients with "intrahepatic" cholangiocarcinoma are excluded from this protocol. This guideline is specifically designed for patients with hilar cholangiocarcinoma or other types of cholangiocarcinoma that are not intrahepatic.

To be eligible for a liver transplant, patients must meet specific inclusion and exclusion criteria. They should be reasonable candidates for liver transplant, have no evidence of extrahepatic disease, including positive nodes, and not have attempted prior resection with violation of the tumor plane. Vascular invasion is a contraindication for transplant, but vascular encasement is not an exclusion criterion.

The mass size is also an essential factor in determining eligibility. When a mass is vis

In [ ]:
# invoke the qa chain and get a response for user query
query = "My patient has portopulmonary hypertension with mean PA Pressure of 50mmHg. Is he eligible for MELD exception points?"
response = qa_chain.invoke({"query": query})
print(response["result"])

**Comprehensive Answer to the Query**

The patient in question has portopulmonary hypertension with a mean PA pressure of 50mmHg. To determine if the patient is eligible for MELD exception points, we need to review the requirements for portopulmonary hypertension MELD score exceptions.

According to the OPTN Policies (Policy 9: Allocation of Livers and Liver-Intestines, Effective Date: 5/29/2024), a candidate will receive a MELD score exception for portopulmonary hypertension if the transplant program submits evidence of all the following:

* Initial mean pulmonary arterial pressure (MPAP) level greater than or equal to 35 mmHg and initial pulmonary vascular resistance (PVR) level greater than or equal to 240 dynes*sec/cm5 (or greater than or equal to 3 Wood units [WU]) via heart catheterization.
* Other causes of pulmonary hypertension have been assessed and determined to not be a significant contributing factor.
* Initial transpulmonary gradient to correct for volume overload.
* Docu

In [ ]:
# invoke the qa chain and get a response for user query
query = "What are the laboratory criteria for a patient to be listed Status 1A  with hepatic artery thrombosis?"
response = qa_chain.invoke({"query": query})
print(response["result"])

**1. Comprehensive answer to the query**

To be listed as Status 1A with hepatic artery thrombosis (HAT), a patient must meet specific laboratory criteria. Hepatic artery thrombosis is a serious complication that can occur after liver transplantation, and prompt re-transplantation is often necessary to prevent graft loss and patient death. The laboratory criteria for listing a patient as Status 1A with HAT are designed to identify patients who are at high risk of graft loss and death, and who require urgent re-transplantation.

According to the guidelines, a patient with HAT within 7 days of transplant can be listed as Status 1A if they meet the following laboratory criteria:

* AST (aspartate aminotransferase) greater than or equal to 3,000 U/L
* At least one of the following:
	+ INR (international normalized ratio) greater than or equal to 2.5
	+ Arterial pH less than or equal to 7.30
	+ Venous pH less than or equal to 7.25
	+ Lactate greater than or equal to 4 mmol/L

These laborato

In [ ]:
# invoke the qa chain and get a response for user query
query = "What are the laboratory criteria for a patient to be listed Status 1A  with hepatic artery thrombosis?"
response = qa_chain.invoke({"query": query})
print(response["result"])

attention_is_all_you_need.pdf


In [ ]:
# invoke the qa chain and get a response for user query
query = "What is the protocol for CMV prophylaxis and monitoring in a D+/R- patient after liver transplant?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer:**

The protocol for CMV prophylaxis and monitoring in a D+/R- patient after liver transplant involves a combination of antiviral medication and regular monitoring of CMV PCR levels. According to the provided guidelines, D+/R- patients are at high risk for CMV infection and disease, and therefore require prophylaxis.

The recommended prophylaxis regimen for D+/R- patients is Valganciclovir, which is administered orally daily for a total course of 3-6 months. However, if the patient experiences side effects such as leukopenia or thrombocytopenia due to Valganciclovir or Ganciclovir, Letermovir may be considered as an alternative. Letermovir is administered at a dose of 480 mg daily, but the dose may need to be adjusted if the patient is also taking Cyclosporine.

In addition to antiviral prophylaxis, regular monitoring of CMV PCR levels is crucial to detect any potential infection or disease. CMV PCR levels should be checked once weekly for 100 days after liver tr

In [ ]:
query = "A patient's native liver explant after transplant shows a large, viable, poorly differentiated hepatocellular carcinoma. There is evidence of microvascular invasion. What is Mayo Clinic's policy on imaging intervals for HCC recurrence monitoring in this patient?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

The patient's native liver explant after transplant shows a large, viable, poorly differentiated hepatocellular carcinoma (HCC) with evidence of microvascular invasion. This information is crucial in determining the patient's risk of HCC recurrence and guiding the post-transplant follow-up care.

According to the Mayo Clinic's Liver Recipient Hepatocellular Carcinoma (HCC) Treatment Guideline, patients with poorly differentiated HCC histology are not classified as low-risk recipients. Instead, they are considered high-risk recipients, although the guideline does not explicitly define the high-risk group. However, it is implied that patients who do not meet the low-risk criteria, including those with poorly differentiated HCC, are at higher risk of recurrence.

Given the patient's high-risk status, the follow-up care should be more intensive than the standard follow-up protocol for low-risk recipients. Although the guideline does not provide specific imaging in

In [ ]:
query = "Outline a detailed week-by-week dietary plan for an obese patient who underwent gastric sleeve at the time of liver transplant."
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

For an obese patient who underwent gastric sleeve surgery at the time of liver transplant, a detailed week-by-week dietary plan is crucial to ensure proper nutrition, weight management, and overall health. The primary goal is to achieve a target weight, with an initial goal of at least 10% weight loss and an ideal goal of achieving a BMI < 30 (Liver Post-Transplant Obesity and Bariatric Surgery Guideline).

In the immediate post-transplant recovery period, the patient may require slightly higher calorie intake to support healing and recovery. However, a calorie-restricted diet with adequate protein intake is essential to promote weight loss and maintain overall health. The Transplant Program Dietitian will determine the target weight and calorie restriction in conjunction with the transplant surgeon.

A sample meal plan for the first few weeks post-transplant may include:

* Week 1-2: Liquid diet with high-calorie, high-protein supplements (e.g., Ensure Clear,

In [ ]:
query = "A 30 year old female underwent liver transplantation. What is the guideline for bone mineral density testing for her?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

According to the Liver Osteoporosis Guideline - Arizona, Florida, Rochester, bone mineral density (BMD) testing is recommended for liver transplant recipients. For a 30-year-old female who underwent liver transplantation, the guideline suggests that BMD testing should be performed at year 1, year 2, and then every 5 years after transplant, assuming she has no risk factors for osteoporosis.

**Key Points or Criteria**

* Age < 40 with no risk factors: BMD testing at year 1, year 2, and every 5 years after transplant
* Risk factors for osteoporosis include:
	+ History of osteoporotic fracture
	+ Z-score < -2 at hip or spine
	+ Significant loss of BMD at hip or spine
	+ On osteoporosis therapy
* If risk factors are present, BMD testing should be performed at year 1, year 2, and every 3 years after transplant

**Relevant Guidelines or Policies**

* Liver Osteoporosis Guideline - Arizona, Florida, Rochester: This guideline provides recommendations for the treatment

In [ ]:
query = "A 40 year old patient had AKI secondary to hepatorenal syndrome and ended up on dialysis. He has not regained urine output and remains on dialysis for the past 2 weeks. Is he eligible for simultaneous liver and kidney transplantation? Please provide the specific eligibility criteria for SLK"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer to the Query**

The patient in question has developed acute kidney injury (AKI) secondary to hepatorenal syndrome and has been on dialysis for the past 2 weeks without regaining urine output. The patient's condition raises the possibility of simultaneous liver and kidney transplantation (SLK). According to the Mayo Clinic's guidelines, renal dysfunction does not exclude a patient from candidacy for liver transplantation. However, the decision to proceed with SLK is made on an individual basis.

In this case, the patient's prolonged dialysis dependence suggests that the kidney dysfunction is severe and may not recover with liver transplantation alone. The patient's condition may meet the criteria for SLK, which is considered for patients with end-stage liver disease and irreversible kidney dysfunction.

**Key Points or Criteria**

* The patient has developed AKI secondary to hepatorenal syndrome and has been on dialysis for the past 2 weeks without regaining urine

In [ ]:
query = "What are rapid recovery pathway recommendations in the OR and ICU after liver transplant?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

The rapid recovery pathway for liver transplant recipients in the operating room (OR) and intensive care unit (ICU) is designed to optimize post-transplant care and minimize complications. According to the Liver Transplant Recipient Rapid Recovery Guideline (Rochester), eligible patients are those who do not meet specific exclusion criteria. The goal of this pathway is to promote a smooth transition from the OR to the ICU and eventually to the general ward, while ensuring close monitoring and timely intervention if needed.

In the OR, the anesthesiologist plays a crucial role in assessing the patient's airway and respiratory status. Patients with a difficult airway or those who require high levels of oxygen therapy (FiO2 > 50%) or positive end-expiratory pressure (PEEP > 8 cm H2O) are excluded from the rapid recovery pathway. Additionally, patients with significant metabolic derangements or acidosis (pH < 7.2, bicarbonate < 18) or those who require more than o

In [ ]:
query = "My patient was diagnosed with intrahepatic cholangiocarcinoma. His tumor size is 2cm. Is he eligible for liver transplantation at Mayo Clinic in the iCCA protocol?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer:**

Based on the provided guidelines, your patient with intrahepatic cholangiocarcinoma (iCCA) and a tumor size of 2cm may be eligible for liver transplantation at Mayo Clinic under the iCCA protocol. The inclusion criteria for liver transplantation in patients with iCCA, as outlined in the Management of Intrahepatic Cholangiocarcinoma (iCCA) in Liver Transplant Candidates and Recipients Guideline, include a single lesion less than or equal to 3 cm in diameter, no vascular invasion, no evidence of extrahepatic disease, and no attempted prior resection with violation of tumor plane.

Since your patient's tumor size is 2cm, which is within the specified limit, and assuming that the other inclusion criteria are met, your patient may be considered for liver transplantation. However, it is essential to note that these guidelines are recommendations rather than rigid rules and may be modified using professional judgment.

**Key Points or Criteria:**

* Single lesion le

In [ ]:
query = "My patient received a hepatitis B core positive liver. What kind of prophylaxis does he need going forward?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer:**

The patient who received a hepatitis B core positive liver requires antiviral prophylaxis to prevent de-novo hepatitis B infection. According to the Mayo Clinic's Liver HBcAb Positive Donor Prevention of de-novo Hepatitis B Guideline, the first dose of antiviral therapy should be initiated on call to the operating room.

The guideline recommends the following antiviral treatment options:

* First choice: Baraclude (entecavir) 0.5 mg oral daily, 2 hours away from food, or Viread (TDF, tenofovir disoproxil fumarate) 300 mg oral daily, with renal adjustment based on serum creatinine levels.
* Second choice: Vemlidy (TAF, tenofovir alafenamide) 25 mg oral daily, with no adjustment needed.

The choice of antiviral therapy depends on the patient's kidney function, with Viread being the preferred option for patients with serum creatinine levels <1.5 mg/dL and no history of osteoporosis. For patients with serum creatinine levels ≥1.5 mg/dL or a history of osteoporosi

In [ ]:
query = "My patient received a liver transplantation. She is CMV seropositive and so was the donor. What kind of post-transplant monitoring for CMV does she need?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer:**

The patient in question is a liver transplant recipient who is CMV seropositive, and the donor was also CMV seropositive. According to the provided guidelines, this patient falls under the category of CMV R+ (recipient seropositive). For such patients, the recommended post-transplant monitoring for CMV involves quantitative blood CMV PCR once weekly for 12 weeks after liver transplantation. No CMV prophylaxis is recommended for these patients; however, they will need HSV prophylaxis.

**Key Points or Criteria:**

* CMV R+ patients require quantitative blood CMV PCR once weekly for 12 weeks after liver transplantation.
* No CMV prophylaxis is recommended for CMV R+ patients.
* HSV prophylaxis is necessary for these patients.
* Preemptive therapy for CMV infection without disease is considered with quantifiable CMV PCR, and treatment should be discussed with Infectious Disease.

**Relevant Guidelines or Policies:**

* Pediatric Liver Transplant Infectious Disea

In [ ]:
query = "What is the Mayo Clinic's recommendation for first and second-line choices for managing hypertension after liver transplant?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

The Mayo Clinic's recommendation for managing hypertension after liver transplant involves a multi-faceted approach that includes lifestyle modifications and pharmacologic interventions. According to the Liver Transplant Hypertension Treatment Guideline, the first-line treatment for hypertension in liver transplant patients is a calcium channel blocker, specifically the dihydropyridine class, such as amlodipine or nifedipine. However, it is essential to note that nifedipine may increase the concentration of calcineurin inhibitors (tacrolimus, cyclosporine), and therefore, increased trough monitoring is recommended.

For patients with specific comorbidities, such as heart failure, active coronary artery disease (CAD), or diabetes, chronic kidney disease (CKD), or significant proteinuria, alternative first-line agents are recommended. For patients with heart failure, beta-blockers or angiotensin receptor-neprilysin inhibitors (ARNI) are preferred, while beta-blo

In [ ]:
query = "My patient's AST spiked to 4000 on day 3 after transplant. Patient is ill with arterial pH of 7.2 and lactate of 5. Ultrasound shows hepatic artery thrombosis. Does he meet the eligibility for Status 1A listing?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

The patient's situation meets the criteria for Status 1A listing due to hepatic artery thrombosis (HAT) within 7 days of transplant. The patient's AST level is significantly elevated at 4000, which is above the required threshold of 3000 U/L. Additionally, the patient's arterial pH is 7.2, and lactate is 5, both of which are within the required criteria for Status 1A listing.

The patient's condition is critical, and the presence of HAT with elevated AST and lactate levels indicates a high risk of graft failure. The patient's arterial pH is also below the normal range, indicating acidosis. These findings are consistent with the diagnosis of HAT and primary non-function of the transplanted liver.

**Key Points or Criteria**

* Hepatic artery thrombosis (HAT) within 7 days of transplant
* AST greater than or equal to 3000 U/L (patient's AST is 4000)
* Arterial pH less than or equal to 7.30 (patient's pH is 7.2)
* Lactate greater than or equal to 4 mmol/L (patien

In [ ]:
query = "A 12 year old boy underwent a liver transplant. The EBV status is D+/R-. What is the recommended monitoring interval for EBV PCR?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer:**

The patient in question is a 12-year-old boy who underwent a liver transplant with an EBV status of D+/R-. According to the Mayo Clinic's guidelines for Post-Transplant Lymphoproliferative Disorder (PTLD) Monitoring & Treatment in Pediatric Liver Transplant Recipients, this patient falls into the high-risk category for developing PTLD.

The recommended monitoring interval for EBV PCR in this patient is every other week during the first 3-6 months post-transplant, based on the immunosuppression load. After this initial period, the monitoring interval should be monthly during the first year post-transplant, and then every 3 months during the second year post-transplant.

**Key Points or Criteria:**

* EBV status: D+/R- (high-risk category)
* Age: 12 years old
* Monitoring interval:
	+ Every other week during the first 3-6 months post-transplant
	+ Monthly during the first year post-transplant
	+ Every 3 months during the second year post-transplant

**Relevant 

In [ ]:
query = "A 60 year old patient is newly diagnosed with multifocal hepatocellular carcinoma. Imaging shows 3 lesions measuring 4cm, 3cm, and 2cm. There are no metastasis or portal vein invasion. AFP is 2000. Is he eligible for MELD exception?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

The patient in question has multifocal hepatocellular carcinoma (HCC) with three lesions measuring 4cm, 3cm, and 2cm, and an alpha-fetoprotein (AFP) level of 2000 ng/mL. To determine eligibility for a Model for End-Stage Liver Disease (MELD) exception, we need to assess whether the patient meets the criteria for a standardized MELD exception for HCC.

According to the OPTN Policy 9 Allocation of Livers and Liver-Intestines (Effective Date: 5/29/2024), a candidate with HCC is eligible for a standardized MELD exception if they meet the T2 stage criteria, which includes having an AFP level less than or equal to 1000 ng/mL. However, in this case, the patient's AFP level exceeds 1000 ng/mL, which means they do not meet the standard T2 stage criteria.

However, the OPTN Policy also provides an exception for candidates with AFP levels greater than 1000 ng/mL. According to the policy, a candidate who has previously had an AFP level greater than 1000 ng/mL at any time 

In [ ]:
query = "My patient has received a liver transplant at another country 10 years ago. He remains on tacrolimus 4mg BID and prednisone 10mg daily. Tacrolimus trough is 8. He has no history of acute cellular rejection as far as he can recall. How should we adjust his medications according to Mayo Clinic's protocol? Be specific."
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

The patient in question has received a liver transplant 10 years ago and is currently on tacrolimus 4mg BID and prednisone 10mg daily, with a tacrolimus trough level of 8. Given that the patient has no history of acute cellular rejection, the primary goal is to adjust the immunosuppressive regimen to minimize long-term side effects while maintaining adequate immunosuppression.

According to the Mayo Clinic's Pediatric Liver Recipient Immunosuppression Guideline (Rochester), the target tacrolimus trough levels for liver transplant recipients are as follows:

* 1st 3 months: 10-12 ng/ml
* Months 3-6: 8-10 ng/ml
* 6-12 months: 5-7 ng/ml
* After 1st year: 3-5 ng/ml

Given that the patient is 10 years post-transplant, the target tacrolimus trough level would be 3-5 ng/ml. The current trough level of 8 is slightly above the target range, suggesting that the tacrolimus dose could be reduced.

Regarding prednisone, the guideline recommends tapering and discontinuing c

In [ ]:
query = "A 46 year old male with acute alcohol-related hepatitis is critically ill with MELD score of 40. This is his first episode of liver decompensation. He has a supportive wife. He does have a history of polysubstance abuse but expresses commitment to lifelong abstinence. What is his eligibility for liver transplant at Mayo Clinic?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer to the Query**

The patient in question, a 46-year-old male with acute alcohol-related hepatitis, is critically ill with a MELD score of 40. This is his first episode of liver decompensation, and he has a supportive wife. Although he has a history of polysubstance abuse, he expresses commitment to lifelong abstinence. To determine his eligibility for liver transplant at Mayo Clinic, we must evaluate his case against the Liver Transplant Acute Alcohol Related Hepatitis Guideline.

The patient meets some of the initial screening criteria, including a clinical diagnosis of alcohol-related hepatitis, a high risk of mortality (MELD score ≥ 20), and a first decompensating event related to liver disease. He also has an identifiable sober support system and expresses commitment to lifelong abstinence. However, his history of polysubstance abuse is a concern, as the guideline specifies "no evidence of polysubstance abuse" as a criterion for consideration.

Despite this, t

In [ ]:
query = "Please provide the complete list of pre-transplant infectious diease evaluation in pediatric patients."
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer:**

The pre-transplant infectious disease evaluation in pediatric patients is a crucial step in the liver transplant process. According to the Pediatric Liver Transplant Infectious Diseases Guideline - Rochester, all liver transplant candidates must undergo a comprehensive serological evaluation to assess their immune status and risk of infectious diseases.

The guideline recommends the following serologies and pre-transplant testing for all liver transplant candidates:

* CMV (IgG only)
* HSV 1/2 (IgG only)
* VZV, Measles, Mumps, and Rubella (IgG only)
* EBV (Antibody profile – EBV VCA IgG/IgM and EBNA)
* HIV 1/2 antibody/antigen screen
* Hepatitis B surface antigen (HBsAg)
* Hepatitis B core antibody (total HBcAb)
* Hepatitis B surface antibody (HBsAb)
* Hepatitis C antibody (HCV Ab)
* Hepatitis C ribonucleic acid (RNA) by nucleic acid test (NAT)
* Syphilis IgG
* Hepatitis A (IgG only)
* Toxoplasma (IgG only)
* Anterior nares swab for MRSA PCR

Additionally, fo

In [ ]:
query = "For liver transplant patients with Roux-en-Y anatomy who undergo a liver biopsy, what are the prophylactic antibiotics of choice?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

**Comprehensive Answer**

For liver transplant patients with Roux-en-Y anatomy who undergo a liver biopsy, the use of prophylactic antibiotics is crucial to prevent sepsis. According to the guidelines, patients with a history of Roux-en-Y or Choledochoduodenostomy anastomosis or prior sphincterotomy are at increased risk of sepsis and require antibiotic therapy.

The recommended antibiotic of choice is ciprofloxacin, which should be administered orally at a dose of 500 mg every 12 hours for 2 doses, starting 1 hour prior to the procedure. If the patient is unable to take oral medication, ciprofloxacin 400 mg intravenous may be used with the same frequency and duration.

**Key Points or Criteria**

* Patients with a history of Roux-en-Y or Choledochoduodenostomy anastomosis or prior sphincterotomy require antibiotic therapy.
* Ciprofloxacin 500 mg oral q 12 hours x 2 doses starting 1 hour prior to procedure is the recommended antibiotic regimen.
* If unable to take oral, ciprofloxacin 4

In [ ]:
query = "For liver transplant patients with Roux-en-Y anatomy who undergo a liver biopsy, what are the prophylactic antibiotics of choice?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])